In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import act
import glob
import matplotlib as mpl
import pyart

In [ ]:
mpl_09 = glob.glob('/home/jhemedinger/dev_act/act-analyses/data/sgp30smplcmask1zwangC1.c1/sgp30smplcmask1zwangC1.c1.2009*')
ceil_09 = glob.glob('/home/jhemedinger/dev_act/act-analyses/data/sgpceilC1.b1/sgpceilC1.b1.2009*')
mpl_09.sort()
ceil_09.sort()

In [ ]:
mpl_09.pop(26)
mpl_09.pop(111)
mpl_09.pop(123)
mpl_09.pop(123)
mpl_09.pop(171)
mpl_09.pop(175)
mpl_09.pop(175)
mpl_09.pop(188)
mpl_09.pop(210)
mpl_09.pop(210)
mpl_09.pop(240)
mpl_09.pop(240)
mpl_09.pop(294)
mpl_09.pop(294)
mpl_09.pop(308)
mpl_09.pop(309)
mpl_09.pop(309)

In [ ]:
ceil_09.pop(26)
ceil_09.pop(111)
ceil_09.pop(123)
ceil_09.pop(123)
ceil_09.pop(171)
ceil_09.pop(175)
ceil_09.pop(175)
ceil_09.pop(188)
ceil_09.pop(210)
ceil_09.pop(210)
ceil_09.pop(240)
ceil_09.pop(240)
ceil_09.pop(294)
ceil_09.pop(294)
ceil_09.pop(308)
ceil_09.pop(309)
ceil_09.pop(309)

In [ ]:
%%time
cbh_mpl_09 = []
time_mpl_09 = []
cbh_ceil_09 = []
time_ceil_09 = []

for mpl in mpl_09:
    mpl_ds = act.io.armfiles.read_netcdf(mpl)
    new_mpl = mpl_ds.resample(time='1min').interpolate('linear')
#     time_mpl_09.append(new_mpl.time.copy().shape)
    nmpl_cbh = np.ma.array(new_mpl.cloud_base.values.copy(), dtype=np.float32)
    nmpl_time = np.ma.array(new_mpl.time.values.copy(), dtype=np.datetime64)
    for i in range(len(nmpl_cbh)):
        cbh_mpl_09.append(nmpl_cbh[i])
        time_mpl_09.append(nmpl_time[i])

for ceil in ceil_09:
    ceil_ds = act.io.armfiles.read_netcdf(ceil)
    new_ceil = ceil_ds.resample(time='1min').interpolate('linear')
#     time_ceil_09.append(new_ceil.time.copy().shape)
    nceil_cbh = np.ma.array(new_ceil.first_cbh.values.copy(), dtype=np.float32)
    nceil_time = np.ma.array(new_ceil.time.values.copy(), dtype=np.datetime64)
    for i in range(len(nceil_cbh)):
        cbh_ceil_09.append(nceil_cbh[i])
        time_ceil_09.append(nceil_time[i])

In [ ]:
mm_09 = np.ma.array(cbh_mpl_09, dtype=np.float32)
cc_09 = np.ma.array(cbh_ceil_09, dtype=np.float32)
tm_09 = np.ma.array(time_mpl_09, dtype=np.datetime64)
tc_09 = np.ma.array(time_ceil_09, dtype=np.datetime64)

mcbh_09 = mm_09[np.ma.where((cc_09 >= 500.) & (cc_09 <= 7620.))[0]]
ccbh_09 = cc_09[np.ma.where((cc_09 >= 500.) & (cc_09 <= 7620.))[0]]
mtime_09 = tm_09[np.ma.where((cc_09 >= 500.) & (cc_09 <= 7620.))[0]]
ctime_09 = tc_09[np.ma.where((cc_09 >= 500.) & (cc_09 <= 7620.))[0]]

a_09 = ccbh_09[np.ma.where((mcbh_09 >= .5) & (mcbh_09 <= 7.620))[0]]
b_09 = mcbh_09[np.ma.where((mcbh_09 >= .5) & (mcbh_09 <= 7.620))[0]]
c_09 = ctime_09[np.ma.where((mcbh_09 >= .5) & (mcbh_09 <= 7.620))[0]]
d_09 = mtime_09[np.ma.where((mcbh_09 >= .5) & (mcbh_09 <= 7.620))[0]]

In [ ]:
print(a_09.min())
print(a_09.max())
print(b_09.min()*1000)
print(b_09.max()*1000)

In [ ]:
print(mcbh_09.shape)
print(ccbh_09.shape)
print(mtime_09.shape)
print(ctime_09.shape)
print(a_09.shape)
print(b_09.shape)
print(c_09.shape)
print(d_09.shape)

In [ ]:
f = (a_09/1000 - b_09)

In [ ]:
fig = plt.figure(figsize=[50,12])
p1 = plt.scatter(c_09, f)
#plt.xlim('20090501', '20090601')
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.title('May 2009 Ceil MPL Height Difference vs Time', fontsize=30)
plt.xlabel('TIME (UTC)', fontsize=25)
plt.ylabel('HEIGHT (m)', fontsize=25)
#plt.ylim(4000, 7262)
#plt.savefig('/home/jhemedinger/dev_act/act-analyses/ceil_mpl_hght_diff_200905.png')
plt.show()
plt.close()

In [ ]:
print(f.mean()*1000)
print(f.std()*1000)

In [ ]:
fig = plt.figure(figsize=[25,25])
plt.scatter(a_09/1000, b_09)
plt.plot(range(0,25), range(0,25), 'k', label='1:1')
plt.plot(np.unique(a_09/1000), np.poly1d(np.polyfit(a_09/1000, b_09, 1))(np.unique(a_09/1000)), 'k--', label='Best Fit Line')
plt.plot(0, 0, 'w', label= ('Mean Difference: ' + '%6.4f km' %(f.mean())))
plt.plot(0, 0, 'w', label= ('STDEV Difference: ' + '%6.4f km' %(f.std())))
plt.xlim(.5, 8)
plt.ylim(.5, 8)
plt.xticks(np.arange(0,8), fontsize=25)
plt.yticks(range(0,8), fontsize=25)
plt.ylabel('MPL Heights (km)', fontsize=30)
plt.xlabel('CEIL Heights (km)', fontsize=30)
plt.title('2009 Ceil vs. MPL Heights', fontsize=30)
plt.legend(loc=4, labelspacing=1, ncol=4, fontsize=18)
plt.savefig('/home/jhemedinger/dev_act/act-analyses/ceil_v_mpl_hghts_2009.png')
plt.show()
plt.close()

In [ ]:
norm = mpl.colors.LogNorm()

In [ ]:
xedges = np.arange(.5, 7.620, 0.25)
yedges = np.arange(.5, 7.620, 0.25)

In [ ]:
fig = plt.figure(figsize=[25,25])
plt.hist2d(a_09/1000, b_09, bins=(xedges, yedges),
           cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
           norm=norm)
plt.plot(range(0,25), range(0,25), 'k', label='1:1')
plt.plot(np.unique(a_09/1000), np.poly1d(np.polyfit(a_09/1000, b_09, 1))(np.unique(a_09/1000)), 'k--', label='Best Fit Line')
plt.plot(0, 0, 'w', label= ('Mean Difference: ' + '%6.4f km' %(f.mean())))
plt.plot(0, 0, 'w', label= ('STDEV Difference: ' + '%6.4f km' %(f.std())))
plt.xlim(.5, 8)
plt.ylim(.5, 8)
plt.xticks(np.arange(0,8), fontsize=25)
plt.yticks(range(0,8), fontsize=25)
plt.ylabel('MPL Heights (km)', fontsize=30)
plt.xlabel('CEIL Heights (km)', fontsize=30)
plt.title('2009 Ceil vs. MPL Heights', fontsize=30)
plt.legend(loc=4, labelspacing=1, ncol=4, fontsize=18)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=25)
plt.savefig('/home/jhemedinger/dev_act/act-analyses/images/ceil_v_mpl_hghts_2d_2009.png')
plt.show()
plt.close()